In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from glob import glob 
from os.path import join as opj
import os

In [13]:
def summarize(df):
    # remove same round correlations
    df = df.loc[df['round_x'] != df['round_y']]
    #df = df.loc[df['trial_x'] != df['trial_y']]
    
    # define trial type
    conditions = [
    (df['pair_x'] != df['pair_y']),
    (df['destination_x'] != df['destination_y']),
    (df['destination_x'] == df['destination_y'])
    ]
    values = ['across', 'within', 'same']
    df['type'] = np.select(conditions, values)
    
    # define valid type
    conditions = [
    (df['valid_x'] != df['valid_y']),
    (df['valid_x'] == True),
    (df['valid_x'] == False)
        ]
    values = ['valid-invalid', 'valid-valid', 'invalid-invalid']
    df['valid'] = np.select(conditions, values)
    
    return df
    
def group(df, li):
    # mean correlations
    df = df.groupby(li)['cor_z'].mean().reset_index()
    df['within_trial_TR'] = df['within_trial_TR_x']
    df = df.drop(columns=['within_trial_TR_x'])
    
    return df
    
rois_dict = {
    'ca23dg-body_thre_0.5_masked':'ca23dg-body',
    'ca1-body_thre_0.5_masked':'ca1-body',
    #'ca23dg_thre_0.5_masked':'ca23dg',
    #'ca1_thre_0.5_masked':'ca1',
    'evc_2_epi_thre_0.5_masked':'evc', 
    'ppa_mni_2_epi_thre_0.5_masked':'ppa'
}

output_dir = "/home/wanjiag/projects/MONSTERA/derivatives/csv_files/python/"
processed_subs = os.listdir(output_dir)

summary_dir = "/home/wanjiag/projects/MONSTERA/derivatives/csv_files/python_summary_z-scores/"
processed_subs = os.listdir(output_dir)

file_dir = []

In [15]:
for k,roi in rois_dict.items():
    print(roi)
    file_dir = glob(opj(output_dir, '*', 'sub-MONSTERA*_rolling3_{}.csv'.format(roi)))
    print(len(file_dir))
    df = pd.concat((pd.read_csv(f) for f in file_dir), ignore_index=True)
    df['cor_z'] = np.arctanh(df['cor'])

    output_df = summarize(df)
    
    summary_df = group(output_df, ['sub_x','type','valid','within_trial_TR_x'])
    summary_df['roi'] = roi
    os.makedirs(opj(summary_dir, roi), exist_ok=True) 
    summary_df.to_csv(opj(summary_dir, roi, '{}_rolling3_summary.csv'.format(roi)), index=False) 

    summary_df = group(output_df, ['sub_x','type','valid','within_trial_TR_x', 'round_x', 'round_y'])
    summary_df['roi'] = roi
    summary_df.to_csv(opj(summary_dir, roi, '{}_rolling3_summary_with_rounds.csv'.format(roi)), index=False) 

    summary_df = group(output_df, ['sub_x','type','valid','within_trial_TR_x', 'pair_x', 'pair_y', 'destination_x', 'destination_y'])
    summary_df['roi'] = roi
    summary_df.to_csv(opj(summary_dir, roi, '{}_rolling3_summary_with_destination.csv'.format(roi)), index=False) 
   
    summary_df = group(output_df, ['sub_x','type','valid','within_trial_TR_x', 'pair_x', 'pair_y', 'round_x', 'round_y'])
    summary_df['roi'] = roi
    summary_df.to_csv(opj(summary_dir, roi, '{}_rolling3_summary_with_pairs_and_rounds.csv'.format(roi)), index=False) 

ca23dg-body
40


/home/wanjiag/projects/environments/glacier_env/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in arctanh
  result = getattr(ufunc, method)(*inputs, **kwargs)


ca1-body
40


/home/wanjiag/projects/environments/glacier_env/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in arctanh
  result = getattr(ufunc, method)(*inputs, **kwargs)


evc
40


/home/wanjiag/projects/environments/glacier_env/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in arctanh
  result = getattr(ufunc, method)(*inputs, **kwargs)


ppa
40


/home/wanjiag/projects/environments/glacier_env/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in arctanh
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
df.head()

,x,y,cor,sub_x,round_x,trial_x,TR_x,pair_x,destination_x,valid_x,...,round_y,trial_y,TR_y,pair_y,destination_y,valid_y,segment_y,within_trial_TR_y,roi,cor_z
0,0,0,1.000000,52,5,0,10,pair3_west,shrub,False,...,5,0,10,pair3_west,shrub,False,same,1,ca23dg-body,inf
1,0,24,-0.129580,52,5,0,10,pair3_west,shrub,False,...,5,1,40,pair3_west,pergola,False,same,1,ca23dg-body,-0.130312
2,0,48,0.034632,52,5,0,10,pair3_west,shrub,False,...,5,2,70,pair3_west,pergola,True,same,1,ca23dg-body,0.034645
3,0,72,0.111274,52,5,0,10,pair3_west,shrub,False,...,5,3,100,pair3_west,shrub,True,same,1,ca23dg-body,0.111737
4,0,96,-0.082171,52,5,0,10,pair3_west,shrub,False,...,5,6,173,pair3_west,shrub,True,same,1,ca23dg-body,-0.082356
